- DECIDE ON DATASET 
- SPATIAL 

# Tutorial

Using ibis + duckdb to do large scale (spatial) data analysis

In [12]:
import ibis as ib
from ibis import _

ib.options.interactive = True

## Introduction

ibis is the link between powerfull backend infrastructure and user friendly `python`. It allows writing code in a familiar environement and with an almost same syntax as `pandas`, but executes the code in a database

In [13]:
conn = ib.connect("duckdb://")

In [14]:
conn.list_catalogs()

['memory', 'system', 'temp']

In [15]:
conn.list_databases()

['information_schema', 'main', 'pg_catalog']

In [16]:
iris = ib.examples.iris_raw.fetch(backend=conn)

In [24]:
iris.schema()

ibis.Schema {
  Sepal.Length  float64
  Sepal.Width   float64
  Petal.Length  float64
  Petal.Width   float64
  Species       string
}

In [32]:
ib.literal(140_000).log()

┌───────────┐
│ 11.849398 │
└───────────┘

In [17]:
flights = ib.examples.nycflights13_weather.fetch(table_name="flights", backend=conn)

In [18]:
flights.describe()

┏━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓
┃ name     ┃ pos   ┃ type    ┃ count ┃ nulls ┃ unique ┃ mode   ┃ mean        ┃ std      ┃ min     ┃ p25     ┃ p50     ┃ p75     ┃ max     ┃
┡━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩
│ string   │ int16 │ string  │ int64 │ int64 │ int64  │ string │ float64     │ float64  │ float64 │ float64 │ float64 │ float64 │ float64 │
├──────────┼───────┼─────────┼───────┼───────┼────────┼────────┼─────────────┼──────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ pressure │    12 │ string  │ 26115 │     0 │    469 │ NA     │        NULL │     NULL │    NULL │    NULL │    NULL │    NULL │    NULL │
│ visib    │    13 │ float64 │ 26115 │     0 │     20 │ NULL   │    9.255372 │ 2.055052 │     0.0 │    10.0 │    10.0 │    10.0 │    10.0 │
│ origin   │     0 │ string  │ 26115 │     0 │      3 │ JFK    │        NULL │     NULL │    NULL │    NULL │    NULL │    NULL │    NULL │
│ year     │     1 │ int64   │ 26115 │     0 │      1 │ NULL   │ 2013.000000 │ 0.000000 │  2013.0 │  2013.0 │  2013.0 │  2013.0 │  2013.0 │
│ month    │     2 │ int64   │ 26115 │     0 │     12 │ NULL   │    6.503733 │ 3.438328 │     1.0 │     4.0 │     7.0 │     9.0 │    12.0 │
│ day      │     3 │ int64   │ 26115 │     0 │     31 │ NULL   │   15.675321 │ 8.762177 │     1.0 │     8.0 │    16.0 │    23.0 │    31.0 │
│ hour     │     4 │ int64   │ 26115 │     0 │     24 │ NULL   │   11.490791 │ 6.912423 │     0.0 │     6.0 │    11.0 │    17.0 │    23.0 │
│ temp     │     5 │ string  │ 26115 │     0 │    174 │ 37.94  │        NULL │     NULL │    NULL │    NULL │    NULL │    NULL │    NULL │
│ dewp     │     6 │ string  │ 26115 │     0 │    154 │ 28.94  │        NULL │     NULL │    NULL │    NULL │    NULL │    NULL │    NULL │
│ humid    │     7 │ string  │ 26115 │     0 │   2500 │ 100    │        NULL │     NULL │    NULL │    NULL │    NULL │    NULL │    NULL │
│ …        │     … │ …       │     … │     … │      … │ …      │           … │        … │       … │       … │       … │       … │       … │
└──────────┴───────┴─────────┴───────┴───────┴────────┴────────┴─────────────┴──────────┴─────────┴─────────┴─────────┴─────────┴─────────┘

In [19]:
# ib.to_sql(flights.describe())

In [20]:
flights_df = flights.execute()

In [30]:
# flights_df.describe()

In [22]:
# conn.list_tables()

## SQL, DUCKDB, IBIS
DuckDB is a *flavour* of SQL, meaning it's generally the same with some extras that are meant to expand or facilitate cetain functionalities that traditional SQL does not have. 

- https://ibis-project.org/tutorials/coming-from/pandas
- 

### Selecting

In [23]:
# SQL
select_sql = conn.sql("""
    SELECT "Petal.Width", Species FROM iris_raw;
""")
select_sql

┏━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Petal.Width ┃ Species ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━┩
│ float64     │ string  │
├─────────────┼─────────┤
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.4 │ setosa  │
│         0.3 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.1 │ setosa  │
│           … │ …       │
└─────────────┴─────────┘

In [39]:
# ibis
select_expr = iris.select("Petal.Width", "Species")
select_expr

┏━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Petal.Width ┃ Species ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━┩
│ float64     │ string  │
├─────────────┼─────────┤
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.4 │ setosa  │
│         0.3 │ setosa  │
│         0.2 │ setosa  │
│         0.2 │ setosa  │
│         0.1 │ setosa  │
│           … │ …       │
└─────────────┴─────────┘

In [40]:
ib.to_sql(select_expr)

```sql
SELECT
  "t0"."Petal.Width",
  "t0"."Species"
FROM "iris_raw" AS "t0"
```

### Filtering

In [41]:
# SQL
filter_sql = conn.sql("""
    SELECT * from iris_raw where "Petal.Width" > .3;
""")
filter_sql

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Sepal.Length ┃ Sepal.Width ┃ Petal.Length ┃ Petal.Width ┃ Species    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ float64      │ float64     │ float64      │ float64     │ string     │
├──────────────┼─────────────┼──────────────┼─────────────┼────────────┤
│          5.4 │         3.9 │          1.7 │         0.4 │ setosa     │
│          5.7 │         4.4 │          1.5 │         0.4 │ setosa     │
│          5.4 │         3.9 │          1.3 │         0.4 │ setosa     │
│          5.1 │         3.7 │          1.5 │         0.4 │ setosa     │
│          5.1 │         3.3 │          1.7 │         0.5 │ setosa     │
│          5.0 │         3.4 │          1.6 │         0.4 │ setosa     │
│          5.4 │         3.4 │          1.5 │         0.4 │ setosa     │
│          5.0 │         3.5 │          1.6 │         0.6 │ setosa     │
│          5.1 │         3.8 │          1.9 │         0.4 │ setosa     │
│          7.0 │         3.2 │          4.7 │         1.4 │ versicolor │
│            … │           … │            … │           … │ …          │
└──────────────┴─────────────┴──────────────┴─────────────┴────────────┘

In [46]:
# ibis
iris.filter(_["Petal.Width"] > .3)

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Sepal.Length ┃ Sepal.Width ┃ Petal.Length ┃ Petal.Width ┃ Species    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ float64      │ float64     │ float64      │ float64     │ string     │
├──────────────┼─────────────┼──────────────┼─────────────┼────────────┤
│          5.4 │         3.9 │          1.7 │         0.4 │ setosa     │
│          5.7 │         4.4 │          1.5 │         0.4 │ setosa     │
│          5.4 │         3.9 │          1.3 │         0.4 │ setosa     │
│          5.1 │         3.7 │          1.5 │         0.4 │ setosa     │
│          5.1 │         3.3 │          1.7 │         0.5 │ setosa     │
│          5.0 │         3.4 │          1.6 │         0.4 │ setosa     │
│          5.4 │         3.4 │          1.5 │         0.4 │ setosa     │
│          5.0 │         3.5 │          1.6 │         0.6 │ setosa     │
│          5.1 │         3.8 │          1.9 │         0.4 │ setosa     │
│          7.0 │         3.2 │          4.7 │         1.4 │ versicolor │
│            … │           … │            … │           … │ …          │
└──────────────┴─────────────┴──────────────┴─────────────┴────────────┘

## IBIS

In [ ]:
iris.describe()

In [ ]:
iris.count()

## Spatial

## Example

### Example 1 : Foursquare

## Ecosystem

Extensions and applications

- OSM data reading
- String matching
- Scalenav
- ....

### Reading OSM buffer files directly into duckdb

In [ ]:
import quackosm # https://github.com/kraina-ai/quackosm

In [ ]:
in_path = '../datasets/OSM/raw/africa-focus.osm.pbf'
out_path = '../datasets/OSM/processed/africa_amenity.parquet'

In [ ]:
# qo.convert_pbf_to_parquet(
#     # pbf_path='../datasets/OSM/raw/africa-latest.osm.pbf',
#     pbf_path=in_path,
#     result_file_path=out_pah,
#     tags_filter={
#         "amenity" : True,
#     },
#     explode_tags=False,
# # )

In [ ]:
res_ = conn.read_parquet(out_path)
res_.head()

In [ ]:
res = (
    res_
    .mutate(
        amenity=_.tags['amenity']
    )
    .select('amenity',"geometry")

)

In [ ]:
res.head()

### Global geospatial anaylytics

In [ ]:
import scalenav.oop as snoo

## Similar tools

polars